In [4]:
import datetime as dt
from calendar import monthrange
import s3fs
import xarray as xr
import pandas as pd
import numpy as np
import os

In [5]:
def get_water_data(sat: str, lyr: int, idyjl: int, band: int = 9):
    """
    Gets GOES water vapor data from AWS S3.

    Args:
        sat: 'goes-east' (GOES-16), 'goes-west' (GOES-17/18)
        yr: Year (2025)
        ldyjl: Julian day (1-365)
        band: ABI band (8=upper-level, 9=mid-level, 10=lower-level)
    
    Return:
        xarray.Dataset of the selected day's water vapor imagery.
    """

    #converts julian data to date-time
    d = dt.datetime(lyr,1,1) + dt.timedelta(days=idyjl - 1)
    #connects to s3 bucket
    fs = s3fs.S3FileSystem(anon=True)
    
    imon,idym=d.month,d.day
    syr,sjdy,smon,sdym = str(lyr).zfill(4),str(idyjl).zfill(3),str(imon).zfill(2),str(idym).zfill(2)
    
    # Choose bucket
    sat_bucket = 'noaa-goes16' if sat == 'goes-east' else 'noaa-goes17'
    sat_bucket = 'noaa-goes16' if sat == 'goes-east' else "noaa-goes19"

    # Path to GOES ABI L2 CMIP (Cloud & Moisture Imagery)
    path_pattern = f"s3://{sat_bucket}/ABI-L2-CMIPF/{syr}/{sjdy}/*/OR_ABI-L2-CMIPF-M6C{str(band).zfill(2)}*.nc"

    # List files and limit
    file_location = fs.glob(path_pattern)[:3]
    var = 'CMI'
    if not file_location:
        raise FileNotFoundError(f"No files found for pattern: {path_pattern}")

    file_ob = [fs.open(file) for file in file_location]        #open connection to files
    
    #open all the day's data
    ds = xr.open_mfdataset(file_ob,combine='nested',concat_dim='time') #note file is super messed up formatting

    ds['BT'] = ds[var]
    ds['BT'].attrs['units'] = 'Kelvin'
    ds['BT'].attrs['long_name'] = 'Brightness Temperature / CMI'
    
    return ds

In [ ]:
regions = {
    "Western Hemisphere": {"x_min": 0, "x_max": 2700, "y_min": 0, "y_max": 5424},
    "Tropics": {"x_min": 1000, "x_max": 4400, "y_min": 2000, "y_max": 3400},
    "North America": {"x_min": 1000, "x_max": 2200, "y_min": 3000, "y_max": 4400},
    "South America": {"x_min": 1200, "x_max": 2600, "y_min": 1000, "y_max": 3000}
}

# Choose satellite automatically
def get_satellite_for_date(date):
    """GOES-16 through April 7 2025 (day 97), then GOES-19."""
    return 'goes-east' if date.timetuple().tm_yday <= 97 else 'goes-19'

daily_csv = "goes16_water_vapor_regions_daily_2025_band_9.csv"
daily_data = []

# If CSV already exists, load it to continue appending
try:
    df_existing = pd.read_csv(daily_csv, parse_dates=["date"])
    last_date = df_existing["date"].max().date()
    start_day = (last_date - dt.date(2025, 1, 1)).days + 2
    daily_data = df_existing.to_dict("records")
    print(f"Starting from {last_date + dt.timedelta(days=1)} (day {start_day})")
except FileNotFoundError:
    start_day = 1

# Collect daily data
for day_of_year in range(start_day, 366):
    date = dt.datetime(2025, 1, 1) + dt.timedelta(days=day_of_year - 1)
    satellite = get_satellite_for_date(date)

    try:
        ds = get_water_data(satellite, 2025, day_of_year, band=8)
    except FileNotFoundError:
        print(f"No data for {date.date()} ({satellite}) — skipping")
        continue

    for region_name, bbox in regions.items():
        region_ds = ds.isel(
            x=slice(bbox["x_min"], bbox["x_max"]),
            y=slice(bbox["y_min"], bbox["y_max"])
        )

        mean_bt = float(region_ds["BT"].mean())
        min_bt = float(region_ds["BT"].min())
        max_bt = float(region_ds["BT"].max())

        record = {
            "date": date.date(),
            "region": region_name,
            "mean_BT": mean_bt,
            "min_BT": min_bt,
            "max_BT": max_bt
        }
        daily_data.append(record)

        # pd.DataFrame(daily_data).to_csv(daily_csv, index=False)

# Make weekly summary
df = pd.DataFrame(daily_data)
df["date"] = pd.to_datetime(df["date"])
df["week"] = df["date"].dt.isocalendar().week

weekly_df = (
    df.groupby(["week", "region"])
      .agg({
          "mean_BT": "mean",
          "min_BT": "min",
          "max_BT": "max"
      })
      .reset_index()
)

weekly_df["week_start_date"] = (
    pd.to_datetime("2025-01-01") + pd.to_timedelta((weekly_df["week"] - 1) * 7, unit="d")
)

# weekly_df.to_csv("goes16_water_vapor_regions_weekly_2025_band_8.csv", index=False)

No data for 2025-11-12 (goes-19) — skipping
No data for 2025-11-13 (goes-19) — skipping
No data for 2025-11-14 (goes-19) — skipping
No data for 2025-11-15 (goes-19) — skipping
No data for 2025-11-16 (goes-19) — skipping
No data for 2025-11-17 (goes-19) — skipping
No data for 2025-11-18 (goes-19) — skipping
No data for 2025-11-19 (goes-19) — skipping
No data for 2025-11-20 (goes-19) — skipping
No data for 2025-11-21 (goes-19) — skipping
No data for 2025-11-22 (goes-19) — skipping
No data for 2025-11-23 (goes-19) — skipping
No data for 2025-11-24 (goes-19) — skipping
No data for 2025-11-25 (goes-19) — skipping
No data for 2025-11-26 (goes-19) — skipping
No data for 2025-11-27 (goes-19) — skipping
No data for 2025-11-28 (goes-19) — skipping
No data for 2025-11-29 (goes-19) — skipping
No data for 2025-11-30 (goes-19) — skipping
No data for 2025-12-01 (goes-19) — skipping
No data for 2025-12-02 (goes-19) — skipping
No data for 2025-12-03 (goes-19) — skipping
No data for 2025-12-04 (goes-19)